In [1]:
from rdflib import Graph
from pydantic import BaseModel
from typing import Optional
import httpx
from pyfuseki import FusekiUpdate

In [5]:
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.content import Content
from api.src.schemas.metadata.bibframe.contribution import Contribution
from api.src.schemas.metadata.bibframe.element import Element
from api.src.schemas.metadata.bibframe.title import Title

class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    content: list[Element]
    language: list[Element]
    title: Title
    classification: Optional[Classification]
    contribution: Optional[list[Contribution]]
    subject: Optional[list[Element]]   
    genreForm: Optional[list[Element]]
    note: Optional[str]
    summary: Optional[str]
    supplementaryContent: Optional[list[Element]]
    illustrativeContent: Optional[list[Element]]
    intendedAudience: Optional[list[Element]]
    tableOfContents: Optional[str]

obj = {
    'adminMetadata': {
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": "22600263"
            }
        ]
    },
    "type": [
        "Work"
    ]
}
# r = Work(**obj)
# r

In [9]:
graph = Graph()
uri = 'http://id.loc.gov/resources/works/13211009'
graph.parse(f'{uri}.rdf')
graph.serialize('old_man.ttl', format='ttl')

<Graph identifier=N58111b51a8df4b1fa9dcfa409155f44a (<class 'rdflib.graph.Graph'>)>

In [2]:
from api.src.function.bibframe.bf_classification import GetClassification
from api.src.function.bibframe.bf_contribution import GetContribution
from api.src.function.bibframe.bf_Uri import GetUriBF
from api.src.function.bibframe.bf_Literal import GetLiteral
from api.src.function.bibframe.bf_title import GetTitle
from api.src.function.bibframe.bf_type import GetType
from api.src.function.bibframe.bf_content import GetContent

def ParserWork(graph, uri):
    identifier = uri.split("/")[-1]
    types = GetType(graph, uri)
    title = GetTitle(graph, uri)

    obj = {
    'adminMetadata': {
        'generationProcess': {
            'label': 'BiblioKeia'
        },
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": identifier
            }
        ]
    },
    "type": types,
    'title': title
    }
    
    # Classification
    obj = GetClassification(graph, uri, obj)
    # Content
    # obj = GetContent(graph, uri, obj)
    # Contribution
    obj = GetContribution(graph, uri, obj)
    # Content
    obj = GetUriBF(graph, uri, 'content', obj)
    # GenreForm
    obj = GetUriBF(graph, uri, 'genreForm', obj)
    # illustrativeContent
    obj = GetUriBF(graph, uri, 'illustrativeContent', obj)
    # intendedAudience
    obj = GetUriBF(graph, uri, 'intendedAudience', obj)
    # language
    obj = GetUriBF(graph, uri, 'language', obj)
    # subject
    obj = GetUriBF(graph, uri, 'subject', obj)
    # supplementaryContent
    obj = GetUriBF(graph, uri, 'supplementaryContent', obj)
    # summary
    obj = GetLiteral(graph, uri, 'summary', obj)
    # tableOfContents
    obj = GetLiteral(graph, uri, 'tableOfContents', obj)

    response = Work(**obj)
    
    return response

response = ParserWork(graph, uri)
response.title
    

NameError: name 'graph' is not defined

MAKE GRAPH

In [2]:
r = httpx.get('http://localhost:8000/import/loc/works?uri=http://id.loc.gov/resources/works/22600263') 
r

<Response [200 OK]>

In [6]:
# from api.src.schemas.metadata.bibframe.work import Work
request = Work(**r.json())
request

Work(adminMetadata=AdminMetadata(encodingLevel='menclvl:f', assigner='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate='2023-06-06', descriptionConventions='http://id.loc.gov/vocabulary/descriptionConventions/isbd', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess=GenerationProcess(label='BiblioKeia', generationDate='2023-06-06T16:15:52'), identifiedBy=[IdentifiedBy(type='Local', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='22600263')], status=Status(value='mstatus:new', label='novo')), type=['Work', 'Text', 'Monograph'], content=[Element(label='text', lang=None, uri='http://id.loc.gov/vocabulary/contentTypes/txt', type=['http://id.loc.gov/ontologies/bibframe/Content'])], language=[Element(label='English', lang='en', uri='http://id.loc.gov/vocabulary/languages/eng', type=['http://id.loc.gov/ontologies/bibframe/Language', 'http://www.loc.go

In [7]:
request.contribution

[Contribution(type=['http://id.loc.gov/ontologies/bflc/PrimaryContribution'], agent='http://id.loc.gov/rwo/agents/n86870523', label='Galor, Oded, 1953-', role='http://id.loc.gov/vocabulary/relators/aut')]

In [19]:
# from api.src.function.bibframe.Work.graphWork import MakeGraphWork
from api.src.function.bibframe.Work.graphWork import MakeClassification, MakeContribution, MakeSubject
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
    """

def MakeGraphWork(request, id):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH bkw:{id}
        {{
                bkw:{id} a { ", ".join([f'bf:{i}' for i in request.type]) }  ;
                bf:adminMetadata [ a bf:AdminMetadata ;
                bflc:encodingLevel {request.adminMetadata.encodingLevel} ;
                bf:assigner <{request.adminMetadata.assigner}> ;    
                bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;    
                bf:descriptionConventions <{request.adminMetadata.descriptionConventions}> ;
                bf:descriptionLanguage <{request.adminMetadata.descriptionLanguage}> ;
                 bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess.label}" ;
                    bf:generationDate "{request.adminMetadata.generationProcess.generationDate}"^^xsd:dateTime ] ;
                bf:identifiedBy [ a bf:Local ;
                    bf:assigner <{request.adminMetadata.assigner}> ;
                    rdf:value "{id}" ] ;
                bf:status {request.adminMetadata.status.value} ] ;
                { MakeClassification(request.classification) if request.classification  else "" }                
                bf:content { ", ".join([f'<{content.uri}>' for content in request.content]) } ;
                bf:language { ", ".join([f'<{language.uri}>' for language in request.language]) } ;
                bf:title [ a bf:Title ;
                bf:mainTitle "{request.title.mainTitle}" 
                { f'; bf:subtitle "{request.title.subtitle}" ' if request.title.subtitle else ''} ] ;
                { MakeContribution(request.contribution) if request.contribution  else "" }  
                { MakeSubject(request.subject) if request.subject  else "" }      
                { f'bf:genreForm { ", ".join([f"<{genreForm.uri}>" for genreForm in request.genreForm ]) } ; '}
                { f'bf:note [ a bf:Note ; rdfs:label "{request.note}" ] ; ' if request.note else '' }
                { f'bf:summary [ a bf:Summary ; rdfs:label "{request.summary.replace('"', )}"^^xsd:string ] ; ' if request.summary else '' }
                { f'bf:tableOfContents [ a bf:tableOfContents ; rdfs:label "{request.tableOfContents}" ] ; ' if request.tableOfContents else '' }
                { f'bf:supplementaryContent { ", ".join([f"<{supplementaryContent.uri}>" for supplementaryContent in request.supplementaryContent ])} ;' if request.supplementaryContent else ''}
                { f'bf:illustrativeContent { ", ".join([f"<{illustrativeContent.uri}>" for illustrativeContent in request.illustrativeContent ])} ;' if request.illustrativeContent else ''}
                { f'bf:intendedAudience { ", ".join([f"<{intendedAudience.uri}>" for intendedAudience in request.intendedAudience ])} ;' if request.intendedAudience else ''} .
        }} }}
        """
    return graph
graph = MakeGraphWork(request, "TESTE")



In [20]:
print(graph)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>    
        
    INSERT DATA {
        GRAPH bkw:TESTE
        {
                bkw:TESTE a bf:Work, bf:Text, bf:Monograph  ;
                bf:adminMetadata [ a bf:

In [35]:
request.summary.replace('"', '')

"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor's gripping narrative explains how technology, population size, and adaptation led to a stunning phase change in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also at an explanation of inequality's ultimate causes: those ancestral populations that enjoyed fruitful geographical characteristics and rich diversity were set on the path to prosperity, while those that lacked it were disadvantaged in ways still echoed today. As we face ecological crisis across the globe, The Jo

In [36]:
request.summary

'"Why are humans the only species to have escaped - only very recently - the subsistence trap, allowing us to enjoy a standard of living that vastly exceeds all others? And why have we progressed so unequally around the world, resulting in the great disparities between nations that exist today? Immense in scope and packed with astounding connections, Galor\'s gripping narrative explains how technology, population size, and adaptation led to a stunning "phase change" in the human story a mere two hundred years ago. But by tracing that same journey back in time and peeling away the layers of influence - colonialism, political institutions, societal structure, culture - he arrives also at an explanation of inequality\'s ultimate causes: those ancestral populations that enjoyed fruitful geographical characteristics and rich diversity were set on the path to prosperity, while those that lacked it were disadvantaged in ways still echoed today. As we face ecological crisis across the globe, T

In [32]:
x = '"Why are humans the '
x.replace('"', '')

'Why are humans the '

In [21]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Lexical error at line 50, column 61.  Encountered: " " (32), after : "Why"\n'

In [25]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = fuseki_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [53]:
g = Graph()
g.parse('https://www.loc.gov/standards/mads/rdf/v1/examples/oca08651008.nt', format='nt')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

In [54]:
g.serialize('EX_NAME.ttl', format='ttl')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

GET WORK

In [45]:
response = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/name/n80002329')
response

<Response [200 OK]>

In [46]:
g = Graph()
g.parse(response.content)
g.serialize('name.ttl', format='ttl')

<Graph identifier=Nc7fd2ec4e1e541edbd048c09e685ea53 (<class 'rdflib.graph.Graph'>)>

In [8]:
g.serialize('bk.rdf', format='xml')

<Graph identifier=Na2c213dac7fc4260b1e9a3ce33f14215 (<class 'rdflib.graph.Graph'>)>

In [51]:
for contribution in request.contribution:
    print(contribution.agent)

https://bibliokeia.com/authorities/name/n80002329


In [42]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = """PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH <{subject}> {{ 
                        <{subject}>  bf:contributionOf   <https://bibliokeia.com/resources/work/{id}> }} }} ; """
for i in request.subject:
    up = subjectOf.format(subject=i.value, id="bk-1")
    response = fuseki_authorities.run_sparql(up)
    print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [32]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = f"""PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH graph:sh85017405 {{ 
                        graph:sh85017405  bf:subjectOf <https://bibliokeia.com/resources/work/bk-2> }} }} ; """
fuseki_authorities.run_sparql(subjectOf)

SOLR

In [40]:
request.genreForm

[GenreForm(label='Ficção', value='genreForms:gf2014026339')]

In [41]:
id = 'bk-1'
doc = {
    'id': id,
    'type': [i.replace("bf:", "") for i in request.type],
    'creationDate': request.adminMetadata.creationDate,
    'status': request.adminMetadata.status.label,
    'cdd': request.classification.classificationPortion,
    'cutter': request.classification.itemPortion,
    'content': [content.label for content in request.content],
    'contribution': [{'id':f'{id}!{contribution.agent.split("/")[-1]}', **contribution.dict()} for contribution in request.contribution],
    'genreForm': [genreForm.label for genreForm in request.genreForm ]
}
doc

{'id': 'bk-1',
 'type': ['Monograph', 'Text', 'Work'],
 'creationDate': '2023-04-20',
 'status': 'novo',
 'cdd': '869.3',
 'cutter': 'M513',
 'content': ['texto', 'Imagem'],
 'contribution': [{'id': 'bk-1!n80002329',
   'type': ['bf:Contribution', 'bflc:PrimaryContribution'],
   'agent': 'https://bibliokeia.com/authorities/name/n80002329',
   'label': 'Machado de Assis',
   'role': 'relators:aut'}],
 'genreForm': ['Ficção']}

In [42]:
solr = pysolr.Solr('http://localhost:8983/solr/acervo/', timeout=10)
r = solr.add([doc], commit=True)
r

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":128}}\n'

In [32]:
solr.delete(q="*:*", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">44</int>\n</lst>\n</response>\n'